<a href="https://colab.research.google.com/github/ben854719/Arctic-BlueSense-AI-Powered-Ocean-Monitoring/blob/main/Agentic_AI_Analyze_Threat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade langchain-google-genai google-generativeai
!pip install --upgrade langchain-google-genai google-generativeai langgraph
!pip install langsmith

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 23.9 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.43.0
    Uninstalling google-auth-2.43.0:
      Successfully uninstalled google-auth-2.43.0
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.55.0
    Uninstalling google-genai-1.55.0:
      Successfully uninstalled google-genai-1.55.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.47.0 which is incompatible.


In [2]:
import os
os.environ["LANGSMITH_API_KEY"] = "LangSmith"

In [3]:
!pip install "mcp[cli]"
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("GeminiTools")

@mcp.tool()
def search(query: str) -> list:
    # Your search logic here
    return ["Result 1", "Result 2"]

In [4]:
!cd mcp-server-demo
!ls

/bin/bash: line 1: cd: mcp-server-demo: No such file or directory
sample_data


In [5]:
!cd mcp-server-demo && uv add langchain-google-genai langgraph

/bin/bash: line 1: cd: mcp-server-demo: No such file or directory


In [14]:
!pip install requests beautifulsoup4

In [31]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from IPython.display import display, HTML
from typing import TypedDict, List
import os
from mcp.server.fastmcp import FastMCP
from google.colab import userdata
from langsmith import traceable
import random
import json
import requests
from bs4 import BeautifulSoup

# Import APl Key.
api_key = userdata.get("Ben856")
if not api_key:
    raise ValueError("Ben856 secret not found. Please set your API key in Colab Secrets.")
os.environ["GOOGLE_API_KEY"] = api_key

# Agent State Definition.

class URLProcessingResult(TypedDict):
   url: str
   extracted_content: str
   recommendation: str

class AgentState(TypedDict):
   input: str
   output: str
   trace_id: str
   context: dict
   url_processing_results: List[URLProcessingResult]

# Prompt Template.

prompt = ChatPromptTemplate.from_template("""
You are a strategic AI agent assisting with for defense and environment issues for Arctic ocean.
Based on the following extracted information from a URL: {url_content}, and the input: {input},
provide recommendations regarding Arctic Ocean vulnerability. Also, respond with modular, secure, and expressive outputs tailored for employer engagement and demo impact.
""")

# Tools Definition.

def telemetry_tool(input: str) -> str:
    return f"Telemetry captured for input: {input}"

def overlay_tool(input: str) -> str:
    return f"Overlay captured for input: {input}"

# Extract information from the website.
def select_url(url: str) -> str:
  try:
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract text from common content areas
    paragraphs = soup.find_all('p')
    text_content = '\n'.join([p.get_text() for p in paragraphs])
    return text_content[:2000]
  except requests.exceptions.RequestException as e:
    return f"Error fetching URL {url}: {e}"
  except Exception as e:
    return f"Error parsing content from {url}: {e}"


# Initialize Gemini Model.
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key=api_key)

# Define a list of target URLs
target_urls = [
    "https://open.canada.ca/data/en/dataset/8ca062f1-edef-48d8-91ac-7ff236ffd9e8/resource/9f60c80f-d689-4783-bd8c-f82f85153283",
    "https://open.canada.ca/data/en/dataset/d4285538-afa5-4644-931a-36d6ab6e25a1",
    "https://www.atlanticcouncil.org/in-depth-research-reports/issue-brief/prioritizing-canadas-investment-in-arctic-infrastructure/"
  ]

# LangGraph nodes.
@traceable(name="entry_node")
def entry_node(state: AgentState) -> AgentState:
    return {**state, "trace_id": f"trace-{random.randint(1000,9999)}"}

@traceable(name="llm_node")
def llm_node(state: AgentState) -> AgentState:
    # Ensure url_processing_results is initialized as a list
    url_processing_results = state.get("url_processing_results", [])

    # Iterate through target URLs
    for url in target_urls:
        url_content = select_url(url)

        # Invoke LLM for recommendation
        response = llm.invoke(prompt.format(input=state["input"], url_content=url_content))

        # Create URLProcessingResult and append to list
        result = URLProcessingResult(
            url=url,
            extracted_content=url_content,
            recommendation=response.content
        )
        url_processing_results.append(result)

    # Update the state with the aggregated results
    # The 'output' can be set to the last recommendation or a summary
    final_output = url_processing_results[-1]["recommendation"] if url_processing_results else "No recommendations generated."
    return {**state, "output": final_output, "url_processing_results": url_processing_results}

@traceable(name="tool_node")
def tool_node(state: AgentState) -> AgentState:
    overlay_output = overlay_tool(state["input"])

    context = {
        "overlay": overlay_output,
        "processed_urls_count": len(state.get("url_processing_results", [])),
    }
    return {**state, "context": context}

@traceable(name="display_node")
def display_node(state: AgentState) -> AgentState:
    html = f"""
        Agent Output
        Response: {state['output']}
        Overlay: {state['context']['overlay']}
        Processed URLs: {state['context']['processed_urls_count']}<br>
        """

    # Display details for each processed URL
    for i, result in enumerate(state['url_processing_results']):
        html += f"""
        <h3>URL {i+1}: <a href="{result['url']}" target="_blank">{result['url']}</a></h3>
        <h4>Extracted Content Snippet:</h4>
        <p>{result['extracted_content'][:300]}...</p>
        <h4>Recommendation:</h4>
        <p>{result['recommendation'][:500]}...</p>
        """

    html += f"""
        <p>Trace ID: {state['trace_id']}</p>
    """
    display(HTML(html))
    return state

# LangGraph workflow.
graph = StateGraph(AgentState)
graph.add_node("entry", entry_node)
graph.add_node("llm", llm_node)
graph.add_node("tools", tool_node)
graph.add_node("display", display_node)

graph.set_entry_point("entry")
graph.add_edge("entry", "llm")
graph.add_edge("llm", "tools")
graph.add_edge("tools", "display")
graph.add_edge("display", END)

# Run agent.
agent_executor = graph.compile()
agent_executor.invoke({"input": "Generate a for Arctic Sea, tailored for stakeholder engagement and technical impact", "url_processing_results": []})

{'input': 'Generate a for Arctic Sea, tailored for stakeholder engagement and technical impact',
 'output': 'As a strategic AI agent for defense and environment issues in the Arctic Ocean, I have synthesized the provided information to generate a focused report and recommendations. The analysis leverages the Atlantic Council\'s methodology of integrating foundational research with concrete policy solutions, applying it to the strategic implications of Canada\'s announced infrastructure investment for Arctic Ocean vulnerability.\n\n---\n\n### **Arctic Ocean Strategic Outlook: Mitigating Vulnerability Amidst Emerging Geopolitical & Economic Shifts**\n\n#### **Executive Summary**\n\nCanada\'s "generational investment" in infrastructure, as outlined in Prime Minister Mark Carney\'s November 2025 budget, signifies a strategic pivot towards enhancing national prosperity and global influence. With significant outlays in pipelines, ports, and roadways, this initiative will inevitably impact th